In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/click-bait/clickbait_titles_0.csv
/kaggle/input/data-transfering/data_0_titles.csv


In [20]:
!pip install transformers[torch] datasets evaluate sacrebleu huggingface-hub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [21]:
import transformers
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import evaluate
from transformers import AutoTokenizer
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq

In [22]:
df = pd.read_csv('/kaggle/input/click-bait/clickbait_titles_0.csv', index_col=0)
df = df[['title', 'clickbait']]

In [23]:
# Удалим дубликаты и пропуски
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [24]:
# Удалим заголовки, на которые YandexGPT не дал ответы
no_answer = 'К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?'
df = df[~df['clickbait'].str.startswith(no_answer)]

# Удалим ненужную информацию из заголовков
def clean(text):
    text = text.strip()
    # Фраза предлагающая один из вариантов кликбейта
    if text.startswith('Вот один из'):
        parts = [part.strip() for part in text.split(':')[1:]]
        text = ' '.join(parts)
    # Фраза предлагающая несколько вариантов кликбейта
    if text.startswith('Вот несколько'):
        text = text.split(':')[1].strip()
    # Ненужные знаки при генерации текста
    useless = ['«', '»', '**', '*']
    for to_change in useless:
        text = text.replace(to_change, '')
    return text

df['clickbait'] = df['clickbait'].apply(clean)
df = df[df['clickbait'].str.len() < 300]

In [25]:
from datasets import Dataset
data_dict = {
    "title": df["title"].tolist(),
    "clickbait": df["clickbait"].tolist()
}

# Создание объекта Dataset
titles = Dataset.from_dict(data_dict)

# Проверка, что датасет успешно загружен
print(titles)

Dataset({
    features: ['title', 'clickbait'],
    num_rows: 18579
})


In [26]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [27]:
titles = titles.train_test_split(test_size=0.1)
titles

DatasetDict({
    train: Dataset({
        features: ['title', 'clickbait'],
        num_rows: 16721
    })
    test: Dataset({
        features: ['title', 'clickbait'],
        num_rows: 1858
    })
})

In [28]:
training_model = 'ai-forever/ruT5-base'
tokenizer = transformers.AutoTokenizer.from_pretrained(training_model)
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(training_model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
def preprocess_function(examples):
    inputs = [example for example in examples['title']]
    targets = [example for example in examples['clickbait']]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=256, truncation=True)
    return model_inputs

tokenized_titles = titles.map(preprocess_function, batched=True)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=training_model)

Map:   0%|          | 0/16721 [00:00<?, ? examples/s]

Map:   0%|          | 0/1858 [00:00<?, ? examples/s]

In [30]:
metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [31]:
from huggingface_hub import notebook_login
notebook_login()

In [33]:
new_model = 'my_t5_small_test'

training_args = Seq2SeqTrainingArguments(
    output_dir=new_model,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_titles["train"],
    eval_dataset=tokenized_titles["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,3.760700,2.360203,4.873800,15.429000
2,2.942200,2.241975,5.373200,15.785300
3,2.635500,2.177653,5.656100,16.129200
4,2.547400,2.138695,5.579300,16.271300
5,2.482300,2.115146,5.854700,16.065700
6,2.389500,2.097186,6.077200,16.306800
7,2.350100,2.079334,5.937400,16.373000
8,2.300900,2.074513,6.061600,16.240000
9,2.287000,2.069530,5.984000,16.367600
10,2.275100,2.066931,5.998500,16.351500


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation 

TrainOutput(global_step=6970, training_loss=2.554496513377645, metrics={'train_runtime': 3586.4489, 'train_samples_per_second': 46.623, 'train_steps_per_second': 1.943, 'total_flos': 4347482400875520.0, 'train_loss': 2.554496513377645, 'epoch': 10.0})

In [34]:
trainer.evaluate(tokenized_titles["test"])

{'eval_loss': 2.0669314861297607,
 'eval_bleu': 5.9985,
 'eval_gen_len': 16.3515,
 'eval_runtime': 33.4939,
 'eval_samples_per_second': 55.473,
 'eval_steps_per_second': 1.762,
 'epoch': 10.0}

In [57]:
header = "Армяне в нарды играют"

from transformers import pipeline
translator = pipeline("text2text-generation", model='nosnic/my_t5_small_test', max_length=256)

In [58]:
translator(header)[0]['generated_text']

'Армяне играют в нарды: что происходит на самом деле?'

In [39]:
texts = ["Исследование влияния погоды на настроение людей",
"Методы улучшения производительности на рабочем месте",
"Анализ структуры и функций клеток",
"Сравнительное исследование различных видов растений",
"Практическое применение основ физики в повседневной жизни",
"Исследование эффективности учебных методик в школах",
"Анализ данных: методы и подходы",
"Влияние факторов окружающей среды на здоровье человека",
"Роль медицинской этики в современной практике",
"Исследование технических характеристик различных моделей автомобилей"]

In [40]:
for text in texts:
    tmp = translator(text)[0]['generated_text']
    display(f'Обычный: {text}')
    display(f'Сгенерированный кликбейт: {tmp}')

'Обычный: Исследование влияния погоды на настроение людей'

'Сгенерированный кликбейт: Погода влияет на настроение людей: как не испортить себе настроение?'

'Обычный: Методы улучшения производительности на рабочем месте'

'Сгенерированный кликбейт: Как повысить производительность рабочего места: советы от эксперта по производительности труда'

'Обычный: Анализ структуры и функций клеток'

...

'Сгенерированный кликбейт: Экологическая среда влияет на здоровье человека: как не навредить себе и не навредить себе?'

'Обычный: Роль медицинской этики в современной практике'

'Сгенерированный кликбейт: Медицинская этика: новая эра в медицине'

'Обычный: Исследование технических характеристик различных моделей автомобилей'

'Сгенерированный кликбейт: Автомобильный бум: как автомобиль изменил ход истории и что скрывает его будущее?'